# Online Outlier Dectection 

In this project, we explore German Credit Dataset which consists of 24 features of 1000 loan applicants and the classification whether each applicant is considered good or bad credit risk. This dataset can be downloaded from UCI Machine Learning Repository: 
https://archive.ics.uci.edu/ml/datasets/Statlog+(German+Credit+Data).

We will approach this problem of identifying bad credit risks using two different unsupervised algorithms: one-class SVM (Support Vector Machine) and DBScan (Density-based Spatial Clustering of Application with noise). We will tweak these algorithms a little to build online outlier detectors. After trainig our models with the initial training set, we will update the models sequentially instead of retraining them from scratch when new sets of data come in. 

Note: As these algorithms are unsupervised, we will assume that we are dealing with unlabled data or data of unknown structure. In other words, we will not use the classification information of samples in training or updating our models. 

## Import Libraries 

We first import libraries that will be used for this project.

In [1]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
# For preprocessig data
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
# For building models 
from sklearn.svm import OneClassSVM
from sklearn.cluster import DBSCAN
from rtree import index
# For building DBScan model 
from rtree.index import Rtree
import math

## Read and split data

We read German credit data, get training set which will be used to initialize models and subsequent test sets which may be used to update models. 

Each dataset consists of 24 features and 100 samples. Besides the initial training set, the model will receive 9 additional datasets. 

In [2]:
# Read the csv file of the data that we will expore. 
os.chdir('./Desktop')
df = pd.read_csv('german_data.csv', header=None)

In [3]:
"""
This dataset consists of 1000 samples, 24 features. 
The last column is classification information: 
1: Considered good credit risk
2: Considered bad credit risk
"""

df.shape

(1000, 25)

In [4]:
# Choose the initial training dataset - first 100 rows whose classification values are 1 (good credit risk)
df_train = df.loc[df[24] == 1].head(100)

In [5]:
# Drop the training set from the original dataset. 
# Shuffle the remaining dataset so that outliers are not too concentrated in the second dataset 
df_rest =df.drop(df_train.index, 0)
df_rest = df_rest.apply(np.random.permutation)

In [6]:
# Divide the training dataset into the set of features and the set of classification values. 
# Only feautres will be used for building models. 
X_train = df_train.iloc[:, 0:24]
y_train = df_train.iloc[:, 24]

In [7]:
# Similarly, divide the remaining dataset into the set of features and the set of classification information.
X_rest = df_rest.iloc[:, 0:24]
y_rest = df_rest.iloc[:, 24]

# 1. One-class SVM 

We first implement a model based on one-class SVM imported from Scikie-learn 
(http://scikit-learn.org/stable/modules/generated/sklearn.svm.OneClassSVM.html).

According to Scikit-learn documentation, one-class SVM has two crucial hyperparameters to be optimized: nu and gamma. 

-nu: an upper bound on the fraction of training errors/ a lower bound on the fraction of support vectors 

-gamma: kernel coefficient for ‘rbf’, ‘poly’ and ‘sigmoid’ 

Ideally, we should tune these hyperparameters using methods such as grid search. For now, the model will use nu=0.1, gamma=0.2 and RBF kernel unless the user feeds in different hyperparameter values. Relatively small value was chosen for nu since the initial training set consists of normal samples most of which should be represented by our model. However, at the same time, we should prevent overfitting. Thus, a relatively small value was chosen for gamma.  


## Overview of the algorithm

-Initialize the model:
1. Standardize features.
2. Reduce the dimensionality of features with principal component analysis (PCA).
3. Fit one-class SVM.

-Update the model:
1. Standardize and compress features of new data with SC and PCA used when preprocessing features of the training data.
2. Given threshold which is 50% of the number of new samples at default,
   
   a)If the number of inliers < threshold, then we don't update the model.

   b)If the number of inliers >= threshold, fit the model with the current model's support vectors and new features.

In [8]:
class one_svm_model(object):
    """One-class SVM classifier
    
    Parameters 
    ----------
    nu: float 
        Lower bound on the fraction of support vectors 
    gamma: flaot
        Kernel coefficient for ‘rbf’, ‘poly’ and ‘sigmoid’
    threshold: int
        Sum of predicted label values, depending on which the model is to be updated or not
        (note: outliers labeled as -1 and inliers labeled as 1)
        
    Attributes
    ----------
    sc_: StandardScaler object 
        Object used to standardize features
        (note: sc_ fitted only once with training data)
    pca_: PCA object 
        Object used for feature extraction
        (note: pca_ fitted only once with training data)
    model_: One-class SVM classifier
    
    """
    
    def __init__(self, nu=0.1, gamma=0.2, threshold=0):
        self.nu = nu
        self.gamma = gamma
        self.threshold = threshold
    
    def model_initialize(self, D_initial_training):        
        """ Initialize model
        
        Parameters
        ----------
        D_initial_training: {array-like}, shape = [n_samples, n_features]
        
        Returns 
        ----------
        self: object
        
        """
        # 1. Standardize features 
        sc = StandardScaler()
        D_std_training = sc.fit_transform(D_initial_training)
        # 2. Compress features with PCA
        pca = PCA(n_components = 2)
        D_pca_training = pca.fit_transform(D_std_training)
        self.sc_ = sc
        self.pca_ = pca
        # 3. Fit one-class SVM
        oneclass_svm = OneClassSVM(nu=self.nu, gamma=self.gamma)
        oneclass_svm.fit(D_pca_training)
        self.model_ = oneclass_svm
        
        return self
    
    def model_update(self, D_new):
        """ Update the model if the majority of new data points are classified as inliers
        
        Parameters
        ----------
        D_new: {array-like}, shape = [n_samples, n_features]
        
        Returns
        ----------
        self: object
        predicted_labels: array (1 for inliers and -1 for outliers) 
        
        """
        # 1. Standardize and compress features
        D_std_new = self.sc_.transform(D_new) 
        D_pca_new = self.pca_.transform(D_std_new)
        
        # 2. Sum up the values of predicted labels
        sum_predicted_values = sum(self.model_.predict(D_pca_new))
        # a) If the sum < threshold, no need to update the current model
        if(sum_predicted_values < self.threshold):
            return self, self.model_.predict(D_pca_new)
        
        # b) Otherwise, update the current model
        else:
            # Use the current model's support vectors and new data to fit the model 
            support_vectors_current = self.model_.support_vectors_.copy()
            D_combined = np.vstack((support_vectors_current, D_pca_new))
    
            self.model_ = self.model_.fit(D_combined)
    
            return self, self.model_.predict(D_pca_new)  

# 2.  DBScan

We now implement another model based on a clustering algorithm, DBScan imported from Scikit-learn (http://scikit-learn.org/stable/modules/generated/sklearn.cluster.DBSCAN.html). 

According to the documentation, DBScan is mostly defined by two hyperparameters: eps and min_samples. 

-eps: the maximum distance between two samples for them to be considered as in the same neighborhood

-min_samples: the number of samples in a neighborhood for a point to be considered as a core point (including the point itself)

Based on these hyperparameters, DBScan assigns each point to one of the three categories:

* core point: a point which has at least a specified number (min_samples) of neighboring points within a specified radius (eps)

* border point: a point that has fewer neighbors than min_samples within eps but is within the raidus eps of a core point

* noise point: a point that is neither core point nor border point

Again, we should optimize hyperparameters using methods such as grid search ideally. For now, their default values are chosen to be eps=0.9, min_samples=4 after some experiments. 

## Overview of the algorithm

-Initialize the model:

1. Standardize features.

2. Reduce the dimensionality of features with principal component analysis (PCA).

3. Fit DBScan.

-Update the model:

1. Standardize and compress features of new data with SC and PCA fit when preprocessing features of the training data.

2. Decide whether to update the model or not

 a) Construct an Rtree which stores the current DBScan model's core points.

 b) For each point in new data, 
 * Find core point(s) closest to the point, querying the Rtree.  
 * If the distance between the point and its closest core point <= eps, then the point is considered as an inlier and otherwise, an outlier. 
 * Assign the predicted label to the point accordingly. 

 c) Given threshold which is 50% of the number of the new samples at default,
 * If the number of inliers < threshold, we don't update the current model
 * Otherwise, we fit the model with the current model's core points and new data points. 
 
Note: In this model, we use Rtree from Scikit-learn to search for neighboring points efficiently. 
(http://toblerity.org/rtree/tutorial.html)

In [9]:
class db_scan_model(object):
    """DBScan 
    
    Parameters 
    ----------
    eps: float 
        the maximum distance between two samples for them to be considered
        as in the same neighborhood 
    min_samples: int
        the number of samples in a neighborhood for a point to be considered 
        as a core point (including the point itself)
    threshold_ratio: float
        the ratio of new samples predicted to be normal, depending on which 
        the model is to be updated or not
        
    Attributes
    ----------
    sc_: StandardScaler object 
        Object used to standardize features
        (note: sc_ fitted only once with training data)
    pca_: PCA object 
        Object used for feature extraction
        (note: pca_ fitted only once with training data)
    model_: DBScan object
    
    """
    def __init__(self, eps=0.9, min_samples=4, threshold_ratio=0.5):
        self.eps = eps 
        self.min_samples = min_samples
        self.threshold_ratio = threshold_ratio
        
    def model_initialize(self, D_initial_training):  
        """ Initialize model
        
        Parameters
        ----------
        D_initial_training: {array-like}, shape = [n_samples, n_features]
        
        Returns 
        ----------
        self: object
        
        """
        # 1. Standardize features
        sc = StandardScaler()
        D_std_training = sc.fit_transform(D_initial_training)
        # 2. Compress features with PCA
        pca = PCA(n_components = 2)
        D_pca_training = pca.fit_transform(D_std_training)
        self.sc_ = sc
        self.pca_ = pca
        # 3. Fit DBScan 
        db = DBSCAN(eps=self.eps, min_samples = self.min_samples, metric='euclidean')
        db = db.fit(D_pca_training)
        self.model_ = db
        
        return self
    
    def model_update(self, D_new):
        """ Update the model if the majority of new data points are classified as inliers
        
        Parameters
        ----------
        D_new: {array-like}, shape = [n_samples, n_features]
        
        Returns
        ----------
        self: object
        predicted_labels: array (-1 for outliers and cluster labels (>= 0) for inliers)
        
        """
        # 0. Prepare values and data structures 
        length_data = D_new.shape[0]
        threshold = length_data * self.threshold_ratio # threshold value (number of predicted inliers)
        pred_labels_current = [] # labels of new data predicted based on the current model (inlier: 1, outlier: -1)
        core_points_current = (self.model_.components_).copy() # core points of the current model
        length_corepoints = core_points_current.shape[0]
        
        # 1. Standardize and compress features 
        D_std_new = self.sc_.transform(D_new) 
        D_pca_new = self.pca_.transform(D_std_new)
        
        # 2. Decide whether to update the current model or not
        # a) Construct an R tree which stores the current model's core points
        temp = np.hstack([core_points_current, core_points_current]) 
        rtree = Rtree()
        for i in range(length_corepoints):
            rtree.add(i, temp[i])
            
        n_inliers = 0 # value that keeps track of the number of inliers in the new dataset 
    
        temp2 = np.hstack([D_pca_new, D_pca_new]) # Create a dummy array to query R tree 
        
        # b) For each new point, find core-point(s) closest to the point
        for j in range(length_data): 
            p = temp2[j]
            index = list(rtree.nearest(p, 1))[0] # Find the index of the first closest core point 
            q = core_points_current[index] # Get the closest core point corresponding to the index 
        
            distance = math.sqrt((D_pca_new[j,0] - q[0])**2 + (D_pca_new[j,1] - q[1])**2)
            
            # If the point is in the neighborhood of a core point, it is considered as an inlier
            if distance <= self.eps:
                n_inliers += 1
                pred_labels_current.append(1)
            # Otherwise, it is considered an outlier                                     
            else:
                pred_labels_current.append(-1)
                
        # c) If the number of inliers is smaller than the threshold, don't update the current model
        if n_inliers < threshold:
            return self, pred_labels_current
        # Otherwise, fit the model with the current model's core points and new data points
        else:
            D_combined = np.vstack([D_pca_new, core_points_current.copy()])
            
            self.model_ = self.model_.fit(D_combined)
            # Create a list of predicted labels for new data: -1 for ouliers and 1 for inliers
            pred_labels_new = []
            for i in self.model_.labels_[0:length_data] :
                if i == -1: 
                    pred_labels_new.append(-1)
                else:
                    pred_labels_new.append(1)
            
            return self, np.asarray(pred_labels_new)    

In [10]:
# Initialize two objects: one_svm_model and db_scan_model 
one_svm_model = one_svm_model()
db_model = db_scan_model()

In [11]:
# Initialize models with the training set
one_svm_model.model_initialize(X_train)
db_model.model_initialize(X_train)

In [12]:
# Update the models sequentially with new test sets
for i in range(8):
    X_test = X_rest.head(100)
    X_rest = X_rest.drop(X_test.index, 0)
    one_svm_model.model_update(X_test)
    db_model.model_update(X_test)

In [13]:
# Update models with the last testset
one_svm_model.model_update(X_test)

(<__main__.one_svm_model at 0x113692550>,
 array([ 1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
         1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
         1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
         1.,  1.,  1., -1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
         1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
        -1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
         1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
         1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.]))

In [14]:
db_model.model_update(X_test)

(<__main__.db_scan_model at 0x113692160>,
 array([ 1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1, -1,  1,  1,
         1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1, -1,  1,
         1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
         1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
         1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
         1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1]))

## References

* Python Machine Learning, written by Sebastian Raschka
* UCI Machine Learning Repository: https://archive.ics.uci.edu/ml/datasets/Statlog+(German+Credit+Data)
* Scikit-learn documentations 
 - One-class SVM: http://scikit-learn.org/stable/modules/generated/sklearn.svm.OneClassSVM.html
 - DBScan: http://scikit-learn.org/stable/modules/generated/sklearn.cluster.DBSCAN.html
 - Rtree: http://toblerity.org/rtree/tutorial.html
* Blog on one-class SVM: http://rvlasveld.github.io/blog/2013/07/12/introduction-to-one-class-support-vector-machines/
* Blog on DBcluster: https://blog.dominodatalab.com/topology-and-density-based-clustering/
* Blog on Rtree and DBCluster: http://stackoverflow.com/questions/35911767/dbscan-with-r-tree-how-it-works
* StackExchange: https://stats.stackexchange.com/questions/30834/is-it-possible-to-append-training-data-to-existing-svm-models